In [2]:
from app.rpd.api import RpdApi, Params
from app.rpd.RpdApp import CompetenceBoard
import json

In [3]:
urls = ["https://rpd.donstu.ru/Rp?rupRowId=3019701&rpId=741082"]
for url in urls:
    url_params = {
        p.split('=', 1)[0]: p.split('=', 1)[1]
        for p in url.split('?', 1)[1].split('&')
    }

    api = RpdApi(data=url_params)
    comps_result = api.get_competencies_of_disciplines()
    
    if comps_result.status_code == 200:
        
        comps = json.loads(comps_result.text)['data']['items']
        cb = CompetenceBoard(None)
        for c in comps:
            cb.add_item(c)
            
        cb.generate_table1(f'./table{api.data[Params.rp_id]}_1.docx')
        cb.generate_table2(f'./table{api.data[Params.rp_id]}_2.docx')
        

GET https://rpd.donstu.ru/Rp3/Load {'rupRowId': '3019701', 'rpId': '741082'}
